

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.



## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
%pip install --upgrade --force-reinstall --no-cache-dir "trl>=0.14,<0.17" unsloth unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 274.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 286.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 302.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 247.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 288.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 269.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 269.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 300.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unsloth-zoo to determine which version is compatible with other requirements. This

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 4096          # Context length (inputs + solution + answer + label)
dtype = torch.bfloat16         # L4 supports bfloat16 natively (good perf / low memory)
load_in_4bit = True            # 4-bit quantization to fit 8B on Colab

# Load base Llama3-8B model + tokenizer
# NOTE:
# - We load the official Unsloth-wrapped Llama 3 8B Instruct weights.
#   ("Instruct" generally gives better reasoning/QA behavior for this task)
# - We are NOT starting from any pre-quantized LoRA checkpoint; this is the base model.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length  = max_seq_length,
    dtype           = dtype,
    load_in_4bit    = load_in_4bit,
)

# --- Put model in train mode (enables grad, LoRA attach later, etc.) ---
FastLanguageModel.for_training(model)

print("Loaded model and tokenizer for training.")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Loaded model and tokenizer for training.


## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **60,000 samples for training** and **5000 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
# Step 3 (Hardened): Data loading, stratified split, balance, formatting

from datasets import load_dataset, Dataset
import numpy as np, random
import math

SEED = 42
random.seed(SEED); np.random.seed(SEED)

#  0) Safety: tokenizer must exist & have pad_token
try:
    _ = tokenizer.encode("ok", add_special_tokens=False)
except NameError as e:
    raise RuntimeError("Tokenizer is not defined. Run the model/tokenizer cell first.") from e

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.padding_side != "right":
    tokenizer.padding_side = "right"  # better with causal LMs + LoRA SFT

print(f"[Tokenizer] eos_token={tokenizer.eos_token!r} | pad_token={tokenizer.pad_token!r} | padding_side={tokenizer.padding_side}")

#  1) Load ONLY the official train split
# trust_remote_code=False to avoid executing custom code from HF repo
full_train = load_dataset(
    "ad6398/nyu-dl-teach-maths-comp",
    split="train",
    trust_remote_code=False,
)

#  2) STRATIFIED train/val split (deterministic 50/50 ratio for VAL)
VAL_SIZE = 5000  # can tune if you want a larger validation
y = np.array([bool(v) for v in full_train["is_correct"]], dtype=bool)
idx_all = np.arange(len(full_train))

true_idx  = idx_all[y]
false_idx = idx_all[~y]

rng = np.random.RandomState(SEED)

# we try to sample VAL_SIZE/2 positives and VAL_SIZE/2 negatives (or as close as possible)
half_val = VAL_SIZE // 2
val_true_sel  = rng.choice(true_idx,  size=min(half_val, len(true_idx)),  replace=False)
val_false_sel = rng.choice(false_idx, size=min(half_val, len(false_idx)), replace=False)

val_idx = np.concatenate([val_true_sel, val_false_sel])
# if we are still short because one class was too small, top up from the other
if len(val_idx) < VAL_SIZE:
    needed = VAL_SIZE - len(val_idx)
    # pick remaining from whichever class still has room
    remain_pool = np.setdiff1d(idx_all, val_idx, assume_unique=False)
    # deterministic top-up
    extra_idx = rng.choice(remain_pool, size=needed, replace=False)
    val_idx = np.concatenate([val_idx, extra_idx])

# final dedupe + sort for reproducibility
val_idx = np.unique(val_idx)
train_idx = np.setdiff1d(idx_all, val_idx, assume_unique=False)

raw_train = full_train.select(train_idx.tolist())
raw_val   = full_train.select(val_idx.tolist())

#  3) Balance TRAIN and cap per class for Colab budget
ys_train = np.array([bool(v) for v in raw_train["is_correct"]], dtype=bool)
idx_true_train  = np.where(ys_train)[0].tolist()
idx_false_train = np.where(~ys_train)[0].tolist()

n_min = min(len(idx_true_train), len(idx_false_train))

# You can raise CAP_PER_CLASS if VRAM/time allows (e.g. 30_000+ each for better accuracy).
# L4 + LoRA 4-bit can usually handle ~40k-60k total steps if you tune batch size / grad_accum.
CAP_PER_CLASS = min(30_000, n_min)

idx_true_train  = idx_true_train[:CAP_PER_CLASS]
idx_false_train = idx_false_train[:CAP_PER_CLASS]

balanced_local_idx = idx_true_train + idx_false_train
# Deterministic shuffle for reproducibility
rng_perm = np.random.RandomState(SEED)
rng_perm.shuffle(balanced_local_idx)

train_balanced = raw_train.select(balanced_local_idx)

#  4) Smart truncation for long solutions (keep head+tail)
# keep more head reasoning to help correctness judgment but still stay under context
HEAD_TOK, TAIL_TOK = 800, 256
EOS = tokenizer.eos_token

def smart_clip(text: str) -> str:
    if text is None:
        return ""
    s = str(text)
    toks = tokenizer.encode(s, add_special_tokens=False)
    if len(toks) <= (HEAD_TOK + TAIL_TOK):
        return s
    head = tokenizer.decode(toks[:HEAD_TOK], skip_special_tokens=True)
    tail = tokenizer.decode(toks[-TAIL_TOK:], skip_special_tokens=True)
    return head + "\n...\n" + tail

# 5) Concise instruction templates
# NOTE: we REMOVE randomness in template choice.
# Stochastic prompting during supervised fine-tuning can inject label noise.
TEMPLATES = [
    (
        "You are a rigorous mathematician. Determine if the provided solution correctly "
        "solves the problem. Reply strictly with 'True' if it is fully correct, "
        "otherwise reply 'False'.\n\n"
        "Question:\n{q}\n\nSolution:\n{s}\n\nAnswer:\n{y}"
    )
]

def lbl(v):
    return "True" if bool(v) else "False"

def format_batch(batch):
    qs, ss, ys = batch["question"], batch["solution"], batch["is_correct"]
    out = []
    for q, s, yv in zip(qs, ss, ys):
        q_str = "" if q is None else str(q)
        s_str = smart_clip("" if s is None else str(s))
        tmpl = TEMPLATES[0]  # deterministic
        out.append(tmpl.format(q=q_str, s=s_str, y=lbl(yv)) + EOS)
    return {"text": out}

# We keep originals intact; only drop columns in the formatted copies.
formatted_train = train_balanced.map(
    format_batch,
    batched=True,
    desc="Formatting balanced train",
).remove_columns([c for c in train_balanced.column_names if c != "text"])

formatted_val = raw_val.map(
    format_batch,
    batched=True,
    desc="Formatting validation",
).remove_columns([c for c in raw_val.column_names if c != "text"])

#  6) Reports
def class_report(ds, name):
    if "is_correct" in ds.column_names:
        yv = [bool(v) for v in ds["is_correct"]]
        t = sum(yv)
        f = len(yv) - t
        ratio = (t / len(yv)) if len(yv) else 0.0
        print(f"{name}: {len(yv)} samples | True={t} False={f} | True ratio={ratio:.3f}")
    else:
        print(f"{name}: no labels column")

print("=== Data summary ===")
class_report(raw_train, "Raw Train (pre-balance)")
class_report(train_balanced, "Train Balanced (used for SFT)")
class_report(raw_val, "Validation (held-out)")

print(f"Formatted sizes → train={len(formatted_train)} | val={len(formatted_val)}")

print("Sample formatted example:\n", formatted_train[0]["text"][:400])


[Tokenizer] eos_token='<|end_of_text|>' | pad_token='<|finetune_right_pad_id|>' | padding_side=right


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Formatting balanced train:   0%|          | 0/60000 [00:00<?, ? examples/s]

Formatting validation:   0%|          | 0/5000 [00:00<?, ? examples/s]

=== Data summary ===
Raw Train (pre-balance): 995000 samples | True=397500 False=597500 | True ratio=0.399
Train Balanced (used for SFT): 60000 samples | True=30000 False=30000 | True ratio=0.500
Validation (held-out): 5000 samples | True=2500 False=2500 | True ratio=0.500
Formatted sizes → train=60000 | val=5000
Sample formatted example:
 You are a rigorous mathematician. Determine if the provided solution correctly solves the problem. Reply strictly with 'True' if it is fully correct, otherwise reply 'False'.

Question:
How many of the positive divisors of 3240 are multiples of 3?

Solution:
We can use sympy to get all divisors and count the ones that are multiples of 3.
<llm-code>
from sympy import divisors

all_divisors = diviso


## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory.


In [ ]:
# Step 4: Configure LoRA (High-capacity variant for accuracy)

# Assumes `model` and `tokenizer` are already loaded and prepped for training.

from unsloth import FastLanguageModel
import torch

# --- LoRA configuration ---
# Higher-capacity LoRA:
# r=32 / lora_alpha=64 gives the adapter more expressive power.
# We keep a small dropout to reduce overfitting on balanced 40k samples.
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,                              # LoRA rank (higher capacity)
    lora_alpha = 64,                     # typically ~2 * r
    lora_dropout = 0.05,                 # keep regularization to avoid overfit spikes
    bias = "none",
    target_modules = [
        "q_proj","k_proj","v_proj","o_proj",   # attention projections
        "gate_proj","up_proj","down_proj",     # MLP projections
    ],
    use_gradient_checkpointing = "unsloth",    # keeps memory in check
    random_state = 42,
)

#  Padding / special tokens safety
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

print("[After fix] eos_token =", tokenizer.eos_token,
      "| pad_token =", tokenizer.pad_token,
      "| pad_id =", tokenizer.pad_token_id,
      "| eos_id =", tokenizer.eos_token_id)

#  Label token sanity check
true_ids  = tokenizer.encode("True",  add_special_tokens=False)
false_ids = tokenizer.encode("False", add_special_tokens=False)

print(f'"True" -> {true_ids} | "False" -> {false_ids}')

if len(true_ids) != 1 or len(false_ids) != 1:
    print("Warning: 'True' and/or 'False' are not single tokens. "
          "We'll handle comparison by string prefix during evaluation.")

print("LoRA configured (r=32, α=64, dropout=0.05) with gradient checkpointing.")


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


[After fix] eos_token = <|end_of_text|> | pad_token = <|end_of_text|> | pad_id = 128001 | eos_id = 128001
"True" -> [2575] | "False" -> [4139]
LoRA configured (r=32, α=64, dropout=0.05) with gradient checkpointing.


### **Second Fine Tune**

In [ ]:
# Step 4 (resume-safe): finalize training mode & sanity checks
from unsloth import FastLanguageModel

# You already ran FastLanguageModel.for_training(model) right after loading.
# Do NOT call get_peft_model() again.

# Pad/eos safety (keeps dataloader happy)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

print("[Tokenizer] eos_token =", tokenizer.eos_token,
      "| pad_token =", tokenizer.pad_token,
      "| pad_id =", tokenizer.pad_token_id,
      "| eos_id =", tokenizer.eos_token_id)

# Label token sanity check (for Step 6/7 scoring path)
true_ids  = tokenizer.encode("True",  add_special_tokens=False)
false_ids = tokenizer.encode("False", add_special_tokens=False)
print(f'"True" -> {true_ids} | "False" -> {false_ids}')
if len(true_ids) != 1 or len(false_ids) != 1:
    print("Note: 'True' and/or 'False' are multi-token — scoring code handles this.")

# Optional: show trainable params to confirm LoRA is active
try:
    from peft import get_peft_model_state_dict
    trainable = sum(p.numel() for n,p in model.named_parameters() if p.requires_grad)
    print(f"Trainable parameters: {trainable:,} (LoRA should be ~1% of total)")
except Exception:
    pass

print("✅ Ready to continue fine-tuning using existing LoRA adapters.")


[Tokenizer] eos_token = <|end_of_text|> | pad_token = <|end_of_text|> | pad_id = 128001 | eos_id = 128001
"True" -> [2575] | "False" -> [4139]
Trainable parameters: 83,886,080 (LoRA should be ~1% of total)
✅ Ready to continue fine-tuning using existing LoRA adapters.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.



## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process.

Grab a coffee, as this will take a few hours\! ☕


In [ ]:
# Step 4 (Version-safe): SFTTrainer configuration

import math, torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

BF16_OK = is_bfloat16_supported()
print(f"bfloat16 supported: {BF16_OK}")

per_device_bs = 34
grad_accum    = 1
effective_bs  = per_device_bs * grad_accum

approx_steps_per_epoch = math.ceil(len(formatted_train) / effective_bs)


steps_per_epoch = math.ceil(40_000 / effective_bs)
target_max_steps = 1412        # 1.2 full epochs

print(f"steps/epoch={steps_per_epoch} | max_steps={target_max_steps} (≈1.2 epochs)")



#  Common kwargs for both modern and legacy TrainingArguments
common_kwargs = dict(
    per_device_train_batch_size = per_device_bs,
    gradient_accumulation_steps = grad_accum,
    warmup_ratio                = 0.05,
    max_steps                   = target_max_steps,
    learning_rate               = 1.2e-4,          # safer LR for LoRA r=32
    weight_decay                = 0.05,
    lr_scheduler_type           = "cosine",
    fp16                        = (not BF16_OK),
    bf16                        = BF16_OK,
    logging_steps               = 50,
    save_strategy               = "steps",
    save_steps                  = 200,
    optim                       = "adamw_torch_fused",    # will use bitsandbytes if available
    max_grad_norm               = 0.3,
    seed                        = 42,
    output_dir                  = "outputs",
    report_to                   = "none",
    remove_unused_columns       = False,
    gradient_checkpointing      = True,        # fine with Unsloth
    group_by_length             = True,        # fewer pads => more stable
    dataloader_num_workers      = 2,
)

# Try modern args with evaluation; if not supported, fall back
try:
    train_args = TrainingArguments(
        **common_kwargs,
        evaluation_strategy      = "steps",
        eval_steps               = 250,
        load_best_model_at_end   = True,
        metric_for_best_model    = "eval_loss",
        greater_is_better        = False,
    )
    use_eval = True
    print("Using TrainingArguments with evaluation_strategy='steps'.")
except TypeError:
    train_args = TrainingArguments(**common_kwargs)
    use_eval = False
    print("Legacy TrainingArguments detected -> proceeding WITHOUT in-train evaluation.")

trainer = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = formatted_train,
    eval_dataset        = (formatted_val if use_eval else None),
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,
    packing             = False,
    args                = train_args,
)

trainer.train()

# If we trained without eval (legacy fallback), at least save the last checkpoint.
if not use_eval:
    trainer.save_model("outputs-last")
    tokenizer.save_pretrained("outputs-last")
    print("Saved last checkpoint to outputs-last (no eval available).")
else:
    # Trainer will have already restored best checkpoint by eval_loss
    trainer.save_model("outputs-best")
    tokenizer.save_pretrained("outputs-best")
    print("✅ Training finished — best checkpoint (by eval_loss) loaded and saved to outputs-best.")


bfloat16 supported: True
steps/epoch=1177 | max_steps=1412 (≈1.2 epochs)
Legacy TrainingArguments detected -> proceeding WITHOUT in-train evaluation.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 40,000 | Num Epochs = 2 | Total steps = 1,412
O^O/ \_/ \    Batch size per device = 34 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (34 x 1 x 1) = 34
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss
50,0.866900
100,0.710200
150,0.704100
200,0.687900
250,0.683600
300,0.675200
350,0.666600
400,0.649600
450,0.654000
500,0.637400


Saved last checkpoint to outputs-last (no eval available).


## **Second Fine Tune**

In [ ]:
# Step 4 (Version-safe): SFTTrainer configuration

import math, torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

BF16_OK = is_bfloat16_supported()
print(f"bfloat16 supported: {BF16_OK}")

per_device_bs = 34
grad_accum    = 1
effective_bs  = per_device_bs * grad_accum

steps_per_epoch  = math.ceil(len(formatted_train) / effective_bs)
target_max_steps = int(steps_per_epoch * 1.5)   # ≈ 1.5 epochs

print(f"steps/epoch={steps_per_epoch} | max_steps={target_max_steps} (≈1.5 epochs)")

max_seq_length = 4096

common_kwargs = dict(
    per_device_train_batch_size = per_device_bs,
    gradient_accumulation_steps = grad_accum,
    warmup_ratio                = 0.05,
    max_steps                   = target_max_steps,
    learning_rate               = 1.1e-4,
    weight_decay                = 0.05,
    lr_scheduler_type           = "cosine",
    fp16                        = (not BF16_OK),
    bf16                        = BF16_OK,
    logging_steps               = 100,
    save_strategy               = "steps",
    save_steps                  = 200,
    optim                       = "adamw_torch_fused",
    max_grad_norm               = 0.3,
    seed                        = 42,
    output_dir                  = "outputs_scale60k",
    report_to                   = "none",
    remove_unused_columns       = False,
    gradient_checkpointing      = True,
    group_by_length             = True,
    dataloader_num_workers      = 2,
)

try:
    train_args = TrainingArguments(
        **common_kwargs,
        evaluation_strategy      = "steps",
        eval_steps               = 200,
        load_best_model_at_end   = True,
        metric_for_best_model    = "eval_loss",
        greater_is_better        = False,
    )
    use_eval = True
    print("Using TrainingArguments with evaluation_strategy='steps'.")
except TypeError:
    train_args = TrainingArguments(**common_kwargs)
    use_eval = False
    print("Legacy TrainingArguments detected -> proceeding WITHOUT in-train evaluation.")

trainer = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = formatted_train,
    eval_dataset        = (formatted_val if use_eval else None),
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,
    packing             = False,
    args                = train_args,
)

trainer.train()

if not use_eval:
    trainer.save_model("outputs_scale60k_last")
    tokenizer.save_pretrained("outputs_scale60k_last")
    print("Saved last checkpoint to outputs_scale60k_last (no eval available).")
else:
    trainer.save_model("outputs_scale60k_best")
    tokenizer.save_pretrained("outputs_scale60k_best")
    print("✅ Training finished — best checkpoint saved to outputs_scale60k_best.")


bfloat16 supported: True
steps/epoch=1765 | max_steps=2647 (≈1.5 epochs)
Legacy TrainingArguments detected -> proceeding WITHOUT in-train evaluation.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60,000 | Num Epochs = 2 | Total steps = 2,647
O^O/ \_/ \    Batch size per device = 34 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (34 x 1 x 1) = 34
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss
100,0.512800
200,0.510000
300,0.510300
400,0.511300
500,0.506200
600,0.501100
700,0.490100
800,0.478200
900,0.479300
1000,0.463100


Unsloth: Will smartly offload gradients to save VRAM!
Saved last checkpoint to outputs_scale60k_last (no eval available).


### **Third Training**

In [ ]:
# CONTINUE FT: ~0.4 epochs with lower LR
import math, torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

BF16_OK = is_bfloat16_supported()
print(f"bfloat16 supported: {BF16_OK}")

per_device_bs = 34
grad_accum    = 1
effective_bs  = per_device_bs * grad_accum

steps_per_epoch = math.ceil(60_000 / effective_bs)  # ≈ 1765 for 60k with bs=34
extra_steps     = 700                                # ≈ 0.4 epochs
print(f"steps/epoch={steps_per_epoch} | extra_steps={extra_steps} (~0.4 epochs)")

OUT_DIR = "outputs_scale60k_cont"

common_kwargs = dict(
    per_device_train_batch_size = per_device_bs,
    gradient_accumulation_steps = grad_accum,
    warmup_steps                = 50,          # tiny warmup for stability
    max_steps                   = extra_steps, # steps in THIS continuation
    learning_rate               = 8e-5,        # lower LR for gentle refinement
    weight_decay                = 0.05,
    lr_scheduler_type           = "cosine",
    fp16                        = (not BF16_OK),
    bf16                        = BF16_OK,
    logging_steps               = 100,
    save_strategy               = "steps",
    save_steps                  = 200,
    optim                       = "adamw_torch_fused",
    max_grad_norm               = 0.3,
    seed                        = 42,
    output_dir                  = OUT_DIR,
    report_to                   = "none",
    remove_unused_columns       = False,
    gradient_checkpointing      = True,
    group_by_length             = True,
    dataloader_num_workers      = 2,
)

# Try with evaluation; if unavailable, it falls back automatically.
try:
    train_args = TrainingArguments(
        **common_kwargs,
        evaluation_strategy      = "steps",
        eval_steps               = 200,
        load_best_model_at_end   = True,
        metric_for_best_model    = "eval_loss",
        greater_is_better        = False,
    )
    use_eval = True
    print("Using TrainingArguments with evaluation_strategy='steps'.")
except TypeError:
    train_args = TrainingArguments(**common_kwargs)
    use_eval = False
    print("Legacy TrainingArguments detected → proceeding WITHOUT in-train evaluation.")

trainer = SFTTrainer(
    model              = model,          # already loaded from /content/outputs_scale60k_last
    tokenizer          = tokenizer,
    train_dataset      = formatted_train,
    eval_dataset       = (formatted_val if use_eval else None),
    dataset_text_field = "text",
    max_seq_length     = 4096,
    packing            = False,
    args               = train_args,
)

trainer.train()

# Save continuation checkpoint
if not use_eval:
    save_dir = OUT_DIR + "_last"
else:
    save_dir = OUT_DIR + "_best"

trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"✅ Continuation checkpoint saved to: {save_dir}")


bfloat16 supported: True
steps/epoch=1765 | extra_steps=700 (~0.4 epochs)
Legacy TrainingArguments detected → proceeding WITHOUT in-train evaluation.


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/60000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60,000 | Num Epochs = 1 | Total steps = 700
O^O/ \_/ \    Batch size per device = 34 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (34 x 1 x 1) = 34
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss
100,0.391500
200,0.366500
300,0.367900
400,0.368500
500,0.362300
600,0.364200
700,0.356200


✅ Continuation checkpoint saved to: outputs_scale60k_cont_last


In [ ]:
#  Step 6 re-eval for continuation checkpoint
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch, torch.nn.functional as F, numpy as np
from tqdm import tqdm

NEW_CKPT = "outputs_scale60k_cont_last"   # change if trainer printed a different folder
BF16_OK = is_bfloat16_supported()
reload_dtype = torch.bfloat16 if BF16_OK else torch.float16
max_seq_length = 4096

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name        = NEW_CKPT,
    max_seq_length    = max_seq_length,
    dtype             = reload_dtype,
    load_in_4bit      = True,
    trust_remote_code = True,
)

# pad/eos safety
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

FastLanguageModel.for_inference(model)
model.eval()
device = next(model.parameters()).device
print("Eval device:", device)

#  reuse your existing Step-6 helpers (smart_clip, build_prompt)
HEAD_TOK, TAIL_TOK = 800, 256

def smart_clip(text: str) -> str:
    s = "" if text is None else str(text)
    toks = tokenizer.encode(s, add_special_tokens=False)
    if len(toks) <= (HEAD_TOK + TAIL_TOK): return s
    head = tokenizer.decode(toks[:HEAD_TOK], skip_special_tokens=True)
    tail = tokenizer.decode(toks[-TAIL_TOK:], skip_special_tokens=True)
    return head + "\n...\n" + tail

INFERENCE_TEMPLATE = (
    "You are a rigorous mathematician. Determine if the provided solution correctly "
    "solves the problem. Reply strictly with 'True' if it is fully correct, "
    "otherwise reply 'False'.\n\n"
    "Question:\n{q}\n\n"
    "Solution:\n{s}\n\n"
    "Answer:\n"
)

def build_prompt(q, s):
    return INFERENCE_TEMPLATE.format(q=("" if q is None else str(q)),
                                     s=smart_clip("" if s is None else str(s)))

t_id = tokenizer.encode("True",  add_special_tokens=False)[0]
f_id = tokenizer.encode("False", add_special_tokens=False)[0]

@torch.inference_mode()
def score_margin(prompt: str) -> float:
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_seq_length).to(device)
    out = model(**enc)
    logprobs = F.log_softmax(out.logits[0, -1], dim=-1)
    return (logprobs[t_id] - logprobs[f_id]).item()

# raw_val must be in memory from Step 3 (rebuild it if you restarted runtime)
margins, golds = [], []
for ex in tqdm(raw_val, desc="Scoring validation set"):
    margins.append(score_margin(build_prompt(ex["question"], ex["solution"])))
    golds.append(bool(ex["is_correct"]))

margins = np.array(margins, dtype=np.float32)
golds   = np.array(golds,   dtype=bool)

cands = np.quantile(margins, np.linspace(0.02, 0.98, 25))
best_acc, best_th = -1.0, 0.0
for th in cands:
    acc = ( (margins >= th) == golds ).mean()
    if acc > best_acc:
        best_acc, best_th = acc, th

print("=======================================")
print(f"Validation size: {len(golds)}")
print(f"Best threshold: {best_th:.6f}")
print(f"Validation accuracy: {best_acc*100:.2f}%")
print("=======================================")


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Eval device: cuda:0


Scoring validation set: 100%|██████████| 5000/5000 [13:36<00:00,  6.13it/s]

Validation size: 5000
Best threshold: -0.375000
Validation accuracy: 86.70%



## **Step 6: Inference and Evaluation**




Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
# Step 6: Inference and Evaluation

import torch, torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from unsloth import FastLanguageModel, is_bfloat16_supported

# 0) Reload the trained model from checkpoint
BF16_OK = is_bfloat16_supported()
reload_dtype = torch.bfloat16 if BF16_OK else torch.float16
max_seq_length = 4096

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name        = "outputs_scale60k_last",
    max_seq_length    = max_seq_length,
    dtype             = reload_dtype,
    load_in_4bit      = True,
    trust_remote_code = True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

FastLanguageModel.for_inference(model)
model.eval()
device = next(model.parameters()).device
print(f"Model reloaded on device: {device}")

# 1) Validation data
validation_dataset = raw_val
print(f"Validation examples: {len(validation_dataset)}")

# 2) Prompt builder (matches training)
HEAD_TOK, TAIL_TOK = 800, 256

def smart_clip(text: str) -> str:
    if text is None:
        return ""
    s = str(text)
    toks = tokenizer.encode(s, add_special_tokens=False)
    if len(toks) <= (HEAD_TOK + TAIL_TOK):
        return s
    head = tokenizer.decode(toks[:HEAD_TOK], skip_special_tokens=True)
    tail = tokenizer.decode(toks[-TAIL_TOK:], skip_special_tokens=True)
    return head + "\n...\n" + tail

INFERENCE_TEMPLATE = (
    "You are a rigorous mathematician. Determine if the provided solution correctly "
    "solves the problem. Reply strictly with 'True' if it is fully correct, "
    "otherwise reply 'False'.\n\n"
    "Question:\n{q}\n\n"
    "Solution:\n{s}\n\n"
    "Answer:\n"
)

def build_prompt(question: str, solution: str) -> str:
    q_str = "" if question is None else str(question)
    s_str = smart_clip("" if solution is None else str(solution))
    return INFERENCE_TEMPLATE.format(q=q_str, s=s_str)

# 3) Label tokens (robust to multi-token cases) + scorer
true_tokens  = tokenizer.encode("True",  add_special_tokens=False)
false_tokens = tokenizer.encode("False", add_special_tokens=False)
single_token_labels = (len(true_tokens) == 1 and len(false_tokens) == 1)

@torch.inference_mode()
def score_example(prompt: str):
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_seq_length).to(device)

    if single_token_labels:
        out = model(**enc)
        last_logits = out.logits[0, -1]
        logprobs = F.log_softmax(last_logits, dim=-1)
        logp_true  = logprobs[true_tokens[0]].item()
        logp_false = logprobs[false_tokens[0]].item()
        return (logp_true - logp_false), logp_true, logp_false
    else:
        def seq_logp(label_ids):
            ids = enc["input_ids"][0].tolist()
            space = max_seq_length - len(ids)
            label_ids = label_ids[:max(space, 0)]
            if not label_ids:
                return float("-inf")
            inp = torch.tensor([ids + label_ids[:-1]], device=device)
            attn = torch.ones_like(inp)
            out = model(input_ids=inp, attention_mask=attn)
            logits = out.logits
            lp = 0.0
            start_pos = inp.shape[1] - (len(label_ids) - 1)
            for t in range(len(label_ids)):
                pos = start_pos + t - 1
                token_id = label_ids[t]
                lp += F.log_softmax(logits[0, pos], dim=-1)[token_id].item()
            return lp

        lp_t = seq_logp(true_tokens)
        lp_f = seq_logp(false_tokens)
        return (lp_t - lp_f), lp_t, lp_f

# 4) Score validation, find threshold, compute accuracy
margins, golds = [], []
for ex in tqdm(validation_dataset, desc="Scoring validation set"):
    prompt = build_prompt(ex["question"], ex["solution"])
    margin, _, _ = score_example(prompt)
    margins.append(margin)
    golds.append(bool(ex["is_correct"]))

margins = np.array(margins, dtype=np.float32)
golds   = np.array(golds,   dtype=bool)

candidate_thresholds = np.quantile(margins, np.linspace(0.02, 0.98, 25))
best_acc, best_th = -1.0, 0.0
for th in candidate_thresholds:
    preds = margins >= th
    acc = (preds == golds).mean()
    if acc > best_acc:
        best_acc, best_th = acc, th

final_preds = margins >= best_th
final_acc = (final_preds == golds).mean()
true_rate_pred = final_preds.mean()
true_rate_gold = golds.mean()

print("=======================================")
print(f"Validation size: {len(golds)}")
print(f"Best threshold: {best_th:.6f}")
print(f"Validation accuracy: {final_acc*100:.2f}%")
print(f"Model 'True' rate after threshold: {true_rate_pred*100:.2f}%")
print(f"Ground truth 'True' rate: {true_rate_gold*100:.2f}%")
print("=======================================")

# 5) One qualitative example
idx = 10
ex = validation_dataset[idx]
prompt = build_prompt(ex["question"], ex["solution"])
margin, lp_t, lp_f = score_example(prompt)
pred_bool = (margin >= best_th)

print("\n#### QUESTION ####")
print(ex["question"])
print("\n#### SOLUTION ####")
print(ex["solution"])
print("\n#### SCORES ####")
print(f"log p('True')  = {lp_t:.6f}")
print(f"log p('False') = {lp_f:.6f}")
print(f"margin(True-False) = {margin:.6f}")
print(f"threshold = {best_th:.6f}")
print("\n#### MODEL PREDICTION ####")
print(pred_bool)
print("\n#### CORRECT ANSWER ####")
print(bool(ex["is_correct"]))


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Model reloaded on device: cuda:0
Validation examples: 5000


Scoring validation set: 100%|██████████| 5000/5000 [13:27<00:00,  6.19it/s]


Validation size: 5000
Best threshold: -0.498047
Validation accuracy: 86.72%
Model 'True' rate after threshold: 50.68%
Ground truth 'True' rate: 50.00%

#### QUESTION ####
A five-digit integer will be chosen at random from all possible positive five-digit integers. What is the probability that the number's units digit will be less than 5? Express your answer as a common fraction.

#### SOLUTION ####
The probability that the units digit will be less than $5$ is equal to the ratio of numbers whose units digit is less than $5$ to the total number of $5$-digit numbers. 
There are $9$ possible units digits less than $5$.
There are $89999$ numbers less than $100000$ whose units digit is less than $5$.
So the answer is $\boxed{\frac{89999}{99999}}$.

#### SCORES ####
log p('True')  = -2.578125
log p('False') = -0.079102
margin(True-False) = -2.499023
threshold = -0.498047

#### MODEL PREDICTION ####
False

#### CORRECT ANSWER ####
False


## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
# Step 7: Generate Submission File (Test Inference → CSV)

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Eval mode / device
model.eval()
device = next(model.parameters()).device
print(f"Inference device: {device}")

# Inference template (must match Step 6)
INFERENCE_TEMPLATE = (
    "You are a rigorous mathematician. Determine if the provided solution correctly "
    "solves the problem. Reply strictly with 'True' if it is fully correct, "
    "otherwise reply 'False'.\n\n"
    "Question:\n{q}\n\n"
    "Solution:\n{s}\n\n"
    "Answer:\n"
)

# Same clipping + sequence length as training/inference
HEAD_TOK, TAIL_TOK = 800, 256
max_seq_length = 4096  # must match Step 6

def smart_clip(text: str) -> str:
    s = "" if text is None else str(text)
    toks = tokenizer.encode(s, add_special_tokens=False)
    if len(toks) <= (HEAD_TOK + TAIL_TOK):
        return s
    head = tokenizer.decode(toks[:HEAD_TOK], skip_special_tokens=True)
    tail = tokenizer.decode(toks[-TAIL_TOK:], skip_special_tokens=True)
    return head + "\n...\n" + tail

def build_prompt(question: str, solution: str) -> str:
    return INFERENCE_TEMPLATE.format(
        q="" if question is None else str(question),
        s=smart_clip("" if solution is None else str(solution)),
    )

# Label tokenization (single- or multi-token safe)
true_tokens  = tokenizer.encode("True",  add_special_tokens=False)
false_tokens = tokenizer.encode("False", add_special_tokens=False)
single = (len(true_tokens) == 1 and len(false_tokens) == 1)

@torch.inference_mode()
def score_margin(prompt: str) -> float:
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_seq_length).to(device)
    if single:
        logits = model(**enc).logits[0, -1]
        lp = F.log_softmax(logits, dim=-1)
        return (lp[true_tokens[0]] - lp[false_tokens[0]]).item()
    else:
        def seq_logp(label_ids):
            ids = enc["input_ids"][0].tolist()
            space = max_seq_length - len(ids)
            label_ids = label_ids[:max(space, 0)]
            if not label_ids:
                return float("-inf")
            inp = torch.tensor([ids + label_ids[:-1]], device=device)
            attn = torch.ones_like(inp)
            logits = model(input_ids=inp, attention_mask=attn).logits
            lp = 0.0
            start = inp.shape[1] - (len(label_ids) - 1)
            for t, tok in enumerate(label_ids):
                lp += F.log_softmax(logits[0, start + t - 1], dim=-1)[tok].item()
            return lp
        return seq_logp(true_tokens) - seq_logp(false_tokens)

# 1) Load official test split
test_dataset = load_dataset(
    "ad6398/nyu-dl-teach-maths-comp",
    split="test",
    trust_remote_code=False,
)
print(f"Test examples: {len(test_dataset)}")

# 2) Predict with tuned threshold from Step 6 (best_th must be defined)
predictions = []
for ex in tqdm(test_dataset, desc="Scoring test"):
    prompt = build_prompt(ex["question"], ex["solution"])
    margin = score_margin(prompt)
    predictions.append(margin >= best_th)

# 3) Build submission
submission = pd.DataFrame({
    "ID": np.arange(len(predictions)),
    "is_correct": predictions,
})

# 4) Save
submission.to_csv("submission.csv", index=False)
print("submission.csv saved.")
print(submission.head())

Inference device: cuda:0
Test examples: 10000


Scoring test: 100%|██████████| 10000/10000 [26:46<00:00,  6.22it/s]

submission.csv saved.
   ID  is_correct
0   0        True
1   1       False
2   2       False
3   3        True
4   4       False


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/drive/MyDrive/llama3_8b_math_verifier_checkpoint


## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint",
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)

FastLanguageModel.for_training(model)

print("✅ Model and tokenizer successfully loaded from checkpoint.")


==((====))==  Unsloth 2025.10.8: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.10.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Model and tokenizer successfully loaded from checkpoint.
